<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignement1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install wandb
import pandas as pd
import numpy as np
import wandb
from keras.datasets import fashion_mnist
from matplotlib import pyplot

In [12]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

(60000, 28, 28)

In [26]:
np.exp(700)[0]

IndexError: ignored

In [ ]:
def feed_forward(X,num_hidden_layers,num_neurons_hl,activation_function,weights,biases,output_function):
    X = X.flatten()

    # X is the input (x1,x2,...,xn)
    # num_neurons_hl is the list of number of neurons in each layer , excluding the input , therefore number of classes should be last element of num_neurons
    # activation_function is just the activation function s.t h_i = activation_function(a_i)
    # weights is a list of n*m where n is number of neurons in previous layer and m is number of neurons in next layer
    # assuming output function O is softmax function
    # biases is list of  [b1,b2,..,bk] for each layer

    h = X
    h = h.reshape((h.shape[0],1))
    a = []
    
    #I will add checks for the num_neurons
    for i in range(num_hidden_layers):
       
       # a_i = W_ih_(i-1) + b_i
       a = np.matmul(W[i],h)+biases[i]
       h = activation_function(a)

    return output_function(a)


def softmax(arr):

    l = []

    flag = False

    cnt = 0

    y = []
    for i in arr : 
      if(i<=700):
         l.append(np.exp(i))
      else:
        flag = True
        cnt = cnt + 1
        y.append(i)

    if(flag):

       temp = np.zeros(shape=(len(arr),1),dtype=float)
       for it in y:
          temp[it][0] = 1.0/cnt;

       return temp   
         
    if(np.sum(l) != 0):
      l = np.array(l)
      l = (l)/np.sum(l)

      l = l.reshape((l.shape[0],1))

       return l

    else

       return (1.0/len(arr))*(np.ones(shape=(len(arr),1),dtype=float))
